In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = 'azure'
openai.api_base = 'https://nous.openai.azure.com/'
openai.api_version = '2023-07-01-preview'
openai.api_key = ''

In [3]:
engine = 'nous'

In [4]:
def predict(text):
    try:
        messages = [
            {"role": "user", "content": text},
        ]
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=0.7,
            max_tokens=1024,
            top_p=0.95,
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [5]:
data = []
with open('paragraph-jurnaldbp.jsonl') as fopen:
    for l in fopen:
        data.append({
            'paragraph': json.loads(l)
        })
        
len(data)

6676

In [6]:
data[0]

{'paragraph': '288                      KANUN     [DISEMBER 2014][ULASAN BUKU]ULASAN BUKU Mohd Bakri IshakUndang-undang dan Pengurusan Sisa Industri di Malaysia dan Eropah, 2013.180 halaman, ISBN9789834610135Pengulas: Shamsuddin Suhor                 sudin@ukm.edu.myFakulti Undang-undang,Universiti Kebangsaan Malaysia,43650 Bangi, Selangor Darul Ehsan, MALAYSIA.Buku ini mengandungi 12 bab yang memperkatakan pelbagai perkara berkaitan dengan sisa industri.  Setiap bab dilihat mampu berdiri dengan sendiri bagi menceritakan sesuatu yang berkaitan dengan sisa, sama ada sisa dari sudut falsafah seperti yang dijelaskan dalam Bab 2, atau sisa industri dari sudut perundangan yang melibatkan Bab 3, 4, 5 dan 6, atau pengurusan sisa industri melalui pendekatan bukan perundangan seperti dalam Bab 9, 10, dan 11.  Seterusnya buku ini membincangkan kes yang menyentuh isu pencemaran alam sekitar secara umum dan/atau yang khusus mengenai sisa industri yang telah diputuskan oleh mahkamah di negara ini s

In [7]:
!mkdir question-jurnaldbp

In [8]:
def generate(filename, l):
    if os.path.exists(filename):
        return
    
    s = f"""
{l['paragraph']}

generate a question related to the context
"""
    s = s.strip()
    r = predict(s)
    l['question'] = r

    with open(filename, 'w') as fopen:
        json.dump(l, fopen)

In [9]:
generate('question-jurnaldbp/0.json', data[0])

In [11]:
from tqdm import tqdm

max_worker = 2
for i in tqdm(range(0, len(data), max_worker)):
    b = data[i: i + max_worker]
    filenames = [(os.path.join('question-jurnaldbp', f'{i + k}.json'), l) for k, l in enumerate(b)]
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, *f): f for f in filenames}

        for future in as_completed(futures):
            future.result()

  6%|██                                    | 184/3338 [04:49<1:21:08,  1.54s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  7%|██▋                                   | 241/3338 [06:18<1:20:35,  1.56s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


  9%|███▏                                  | 285/3338 [07:30<1:19:37,  1.56s/it]

...

 98%|████████████████████████████████████▎| 3278/3338 [1:29:49<01:47,  1.79s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 99%|████████████████████████████████████▋| 3308/3338 [1:30:44<00:52,  1.76s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


100%|█████████████████████████████████████| 3338/3338 [1:31:32<00:00,  1.65s/it]
